In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import base64
import string
import re
from collections import Counter
import sklearn
import xlrd
import nltk as nltk
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from os import path
from PIL import Image

In [ ]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import thinc.extra.datasets
import pandas as pd
import spacy
from spacy.util import minibatch, compounding
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from numpy import array, unique, array_equal
import keras
max_fatures = 3500
from keras import regularizers


def load_data():
    print("Loading Data...")
    TwitterEmotion = pd.read_csv('text_emotion.csv')
    print("Data Loaded")
    return (TwitterEmotion['content'], array(TwitterEmotion['sentiment']))


#Below the data is sorted into nine emotion groups. Eight of the groups are the outter layer of the wheel, or the combinations of 
#two emotion groups. The ninth group is "Ambiguous" and "Neutral" put together.
def sort_to_2_emotions(sentence_list, emotion_list):
    sorted_list = []
    sorted_emo = []
    for (data, emo) in zip(sentence_list, emotion_list):
        if (emo == 'enthusiam' or emo == 'love' or emo == 'happiness'):
            sorted_list.append(data)
            sorted_emo.append("positive")
        if (emo == 'sadness' or emo == 'hate'):
            sorted_list.append(data)
            sorted_emo.append("negative")
    return (sorted_list, sorted_emo)



def tokenize(sentences):
    print("Tokenizing...")
    tokenizer = Tokenizer(num_words=max_fatures, split=' ',lower=True)
    tokenizer.fit_on_texts(sentences)
    X = tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(X)
    print("Tokenization Completed")
    return X



def train(tokened_sentences, emotion_list):
    print("Training...")
    embed_dim = 128
    lstm_out = 196
    test_percent=.1

    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = tokened_sentences.shape[1]))
    model.add(SpatialDropout1D(0.7))
    model.add(keras.layers.Dropout(.6))

    model.add(LSTM(lstm_out, dropout=0.65, recurrent_dropout=0.65))
    
    # len(set(emotion_list)) is a hacky way of geting the number of unique elements
    # in a regualar python list (non-numpy)
    model.add(Dense(unique(emotion_list).size,activation='softmax'))
    model.compile(loss = 'binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=True ),metrics = ['accuracy'])
    #print(model.summary())

    Y = pd.get_dummies(emotion_list).values
    X_train, X_test, Y_train, Y_test = train_test_split(tokened_sentences,Y, test_size = test_percent, random_state = 152321326)
    #print(X_train.shape,Y_train.shape)
    #print(X_test.shape,Y_test.shape)

    batch_size = 32
    model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1,validation_split=.0)
    print("Training Completed")
    print("Testing Against Control... (% of the data) ", test_percent)
    score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
    print("Score   :", score)
    print("Accuracy:", acc)
    return (model, X_test)

### When Splitting data (train_test_split), we don't retain where in the 
### origional set the data is located, thus it takes
### a little trickery to see the results while while only testing against untrained data
### top_predictions variable changes how many predictions given
def test(model, X_test, tokenized_data, sentence_list, emotion_list, top_predictions=1):
    predictions = model.predict(tokenized_data, batch_size=32)
    error = 0
    for i in range(len(predictions)):
     
        #  "If the current tokenized data array is in X_test (untrained tokenized arrays)
        
        if(any(array_equal(tokenized_data[i], x) for x in X_test)):
            print("\n\n")

            
            pos = list(predictions[i]).index(max(predictions[i]))
            if (unique(emotion_list)[pos] != emotion_list[i]):
                error += 1
            temp = predictions[i]
            
            print(sentence_list[i])

            for j in range(top_predictions):
                pos = list(temp).index(max(temp))
                print("\n # %s Predicted emotion : ",j+1, unique(emotion_list)[pos])
                temp[pos] = 0
            print("Actual emotion   : ", emotion_list[i])

    print("%  Accuracy when against untrained set: ", 1- (float(error) / len(X_test)))

#################################
#############MAIN################
#################################
#   jupyter notebook is weird   #




(sentence_list, emotion_list) = load_data()


### Comment the Below line for all 18 emotions. This sorts into "positive" and "negative"
sentence_list, emotion_list = sort_to_2_emotions(sentence_list, emotion_list)

tokenized_data = tokenize(sentence_list)

(model, X_test) = train(tokenized_data, emotion_list)

test(model, X_test, tokenized_data, sentence_list, emotion_list)


Loading Data...
Data Loaded
Tokenizing...
Tokenization Completed
Training...
Epoch 1/10
13985/13985 [==============================] - 58s 4ms/step - loss: 0.6596 - acc: 0.6013
Epoch 2/10
13985/13985 [==============================] - 58s 4ms/step - loss: 0.5843 - acc: 0.6875
Epoch 3/10
13985/13985 [==============================] - 56s 4ms/step - loss: 0.5321 - acc: 0.7316
Epoch 4/10
13985/13985 [==============================] - 52s 4ms/step - loss: 0.5033 - acc: 0.7545
Epoch 5/10
13985/13985 [==============================] - 63s 5ms/step - loss: 0.4788 - acc: 0.7731
Epoch 6/10
13985/13985 [==============================] - 52s 4ms/step - loss: 0.4527 - acc: 0.7934
Epoch 7/10
13985/13985 [==============================] - 51s 4ms/step - loss: 0.4357 - acc: 0.7986
Epoch 8/10
13985/13985 [==============================] - 52s 4ms/step - loss: 0.4166 - acc: 0.8109
Epoch 9/10
13985/13985 [==============================] - 54s 4ms/step - loss: 0.4034 - acc: 0.8188
Epoch 10/10
13985/13985




@WoWInsider I would be playing my old rogue but a friend took the account and could not get it back

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



Can I get tonight's SYTYCD online anywhere...streamed like Hulu or paid like iTunes. I missed my favorite judge and now I can't sleep.

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



Time to do house chores! Ugh!

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



where's enthusiasm in meeeee

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



i'm so sick   bad throat and the WORST toothache.  good thing i work in a pharmacy.  load me up with drugs!!

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



roughnight

 # %s Predicted emotion :  1 positive
Actual emotion   :  negative



@stevecruzblog  It is so unfair, Hustlaball, US citizens working here, Personal Appearances Expectations, Prowler, can I do that there?

 # %s Predicted emot




@Kellyke28 You poor thing  Hang in there. -xxx- luf joe

 # %s Predicted emotion :  1 negative
Actual emotion   :  positive



trying to get used to twitter, i have a cold

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



Archie Proposes Veronica in the Latest Archie's Comic. The Longest known Love Triangle is coming to an End. Poor Betty.

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



At school, it's so boring

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



itï¿½s raining... I hate rain...  going to shop and drink coffee all day.

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



oowww...why do wisdom teeth hurt so much

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



hate this fucking cold. i can't stop sneezing. and claratyne only works for an hr or 2

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



Sick, sick, sick. This sucks. i can't even breathe 




@1outside a jo; tak nic no

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@woodywoodford unfortunately I didn't leave the house in time

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



feels so used I hate guys grrrr

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@erinealberty that sucks!

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



My storm is acting up  ....Excited for the discussion session regarding Social Media. Scott Lake, CEO of ThinkSM, will be attending.

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



@chevale my internet is being a ***** again!!! wtf. i think im gonna sleep afterward. internet is driving me crazyyy!

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@HBSExecEd that was the class I was keen on. Guess I have to wait for next summer. bummer.

 # %s Predicted emotion :  1 positive
Actual emotion   :  negative



@BrittanyAS




i realy wanted to go out cause its so nice but everybodys busy

 # %s Predicted emotion :  1 negative
Actual emotion   :  positive



@AnnieAreYouOhK No, it's annoying and makes us girls that follow sports and are knowledgable look dumb

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



I need some cheering up

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



my blow torch has broken  this is a REAL bummer

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@Spiffums I wasn't near him.

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@CapAtomyc That link isn't working for me for some reason

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@roxeecruz ur linky no worky

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



listening through the Mendeed discog once again. I still think their disbanding is a loss.  #metal

 # %s Predicted emotion :  1 positive
Actual e

...




I was hoping to sleep in tomorrow ..I guess it's not happening  Stupid blood tests grrr..

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



When youre kawawa, you make me kawawa cuz i hate seeing you so kawawa's  ohhhhh &lt;3

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



I actually got a bit of a tan today haha sadly will be working all weekend won't get a chance to enjoy the glorious sunrays boo

 # %s Predicted emotion :  1 negative
Actual emotion   :  positive



So I thought the Cavs would crush the Magic and I come home from up north to realize how wrong I was.

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@nkirchmar I have 30 minutes to go! Then a 3 day weekend leading up to my birthday on Tuesday. I have to work on my birthday.

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



Last day of holydays, gotta get back to work    Anyway, 2 great weeks

 # %s Predicted emotion :  1 negative
A




I just want to watch my Naruto Shippuuden missed episodes.

 # %s Predicted emotion :  1 negative
Actual emotion   :  positive



Has a head cold

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



You such a good friend &lt;3 I miss you so much

 # %s Predicted emotion :  1 positive
Actual emotion   :  negative



FML.. today sucks.. i just hope the dance will bring my soul up.. i pray, but im still sad..i hateee todayyyyyy!!! &gt;;(

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



aw &amp; I really miss Alice &amp; Charlotte tonight  Dunno why though, hey are both spoons ;)

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@jaidenkhat sadface

 # %s Predicted emotion :  1 positive
Actual emotion   :  negative



@RealAudreyKitch i'd seriously kill for a bath the now.  but we only have a shower cubicle. I miss bubble baths!

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



I'm so hungry

 # %s Predic




@bjolly086 Now, I can't call your kitten by its new name, because Ryan makes fun of the way I say kitten.

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



@aimzy24 I'm sorry, but x-men sucks... in a bad way!  ;p

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



... I want to die. I want to TAKE my own life, forever. She's trying to get me to do a duet.

 # %s Predicted emotion :  1 negative
Actual emotion   :  positive



@Lady_Lynn @deanna_raquel ahh man  ok if y'all end up doin sumthin else let me kno so I can meet y'all 2 see wht yal gone do..

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



youtube not working

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



i miss you

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



I'm very upset right now, like I can't even formulate a complete thought....

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



In pain,,




NEW LOGO! for all the Web, Cheak it!...gAllethOo

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



good job!

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



I was crying all of Tuesday cos I find out! ;( .... Economics is offically my crying subject lol

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



Just hangin out at home, watchin twiligghhttt ( and readin it (AGAIN!!) lol School sukked today Bahaha

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



@williamduncan the archetype thing was for a comm. class that studied archetypes in popular media. reading your links now, thanks

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



@RaspberryHatter  aw. that sounds amazing  But I think I have work. Thank you for inviting me though

 # %s Predicted emotion :  1 positive
Actual emotion   :  negative



@galacticfaerie Sounds great. I'm looking forward to it then

 # %s Predicted emo




is super happy at the new interest rate.  whoop whoop.

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



@Ciaee Yessir, yessir, three cheeks full.

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



Boys are stupid, throw rocks at them

 # %s Predicted emotion :  1 negative
Actual emotion   :  negative



@joyee0720 Thanks PP!

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



@ralphp eagerly anticipating twitpics

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



@LittleFletcher Oh, and Happy Judd Day haha  #juddday

 # %s Predicted emotion :  1 positive
Actual emotion   :  positive



Ok so this shower has been calling my name for about an hour I think I'll indulge &amp; respond now while I allow Robin Thicke to serenade me

 # %s Predicted emotion :  1 negative
Actual emotion   :  positive



@gootecks you played some really tight matches man, good shit. constantly evolvin, never boring. fuck east coa

# fig = plt.figure(figsize=(20,4))
sns.barplot(x = ProjectData['emotion'].unique(), y=ProjectData['emotion'].value_counts())
plt.show()